## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-01-14-53-58 +0000,bbc,US and UK agree zero tariffs deal on pharmaceu...,https://www.bbc.com/news/articles/cn0k520v4xro...
1,2025-12-01-14-52-37 +0000,nypost,"Bitcoin craters below $86,000, Ethereum dips 7...",https://nypost.com/2025/12/01/business/bitcoin...
2,2025-12-01-14-52-00 +0000,wsj,Australia’s SecondQuarter Raising Third Fund T...,https://www.wsj.com/articles/australias-second...
3,2025-12-01-14-51-48 +0000,nyt,Zelensky Turns to Europe After Talks With U.S....,https://www.nytimes.com/2025/12/01/world/europ...
4,2025-12-01-14-51-00 +0000,wsj,Dollar Weighed Down by Fed Chair Speculations,https://www.wsj.com/finance/currencies/usd-jpy...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2314/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
34,trump,28
63,new,11
305,party,9
304,shooting,8
383,up,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
10,2025-12-01-14-38-59 +0000,nypost,Trump backs Hegseth after report claims war se...,https://nypost.com/2025/12/01/us-news/trump-ba...,70
77,2025-12-01-11-00-00 +0000,latimes,"Attacks on ICE up 1,000%? Trump administration...",https://www.latimes.com/california/story/2025-...,64
137,2025-12-01-04-00-00 +0000,wsj,Economists predicted a global shock from Presi...,https://www.wsj.com/economy/trade/americas-tar...,61
172,2025-11-30-22-28-52 +0000,nypost,Why Venezuela’s allies Russia and China are sl...,https://nypost.com/2025/11/30/world-news/why-v...,61
8,2025-12-01-14-42-17 +0000,nypost,Appeals court disqualifies former Trump lawyer...,https://nypost.com/2025/12/01/us-news/appeals-...,57


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
10,70,2025-12-01-14-38-59 +0000,nypost,Trump backs Hegseth after report claims war se...,https://nypost.com/2025/12/01/us-news/trump-ba...
44,47,2025-12-01-12-41-27 +0000,bbc,Three children among four killed in shooting a...,https://www.bbc.com/news/articles/cvg81p61dnmo...
103,42,2025-12-01-09-57-37 +0000,cbc,"Death toll rises, hundreds missing as Indonesi...",https://www.cbc.ca/news/world/death-toll-rises...
73,37,2025-12-01-11-00-00 +0000,nypost,Jersey City mayoral run-off between ex-NJ Gov....,https://nypost.com/2025/12/01/us-news/jersey-c...
77,36,2025-12-01-11-00-00 +0000,latimes,"Attacks on ICE up 1,000%? Trump administration...",https://www.latimes.com/california/story/2025-...
154,36,2025-12-01-01-18-50 +0000,nypost,"Female pilot, Navy officer flight student pres...",https://nypost.com/2025/11/30/us-news/female-p...
129,34,2025-12-01-05-28-53 +0000,nyt,Hong Kong’s Migrant Domestic Workers Mourn The...,https://www.nytimes.com/2025/11/30/world/asia/...
89,28,2025-12-01-10-30-00 +0000,wsj,The coming interest-rate decision by the Feder...,https://www.wsj.com/economy/central-banking/fe...
137,27,2025-12-01-04-00-00 +0000,wsj,Economists predicted a global shock from Presi...,https://www.wsj.com/economy/trade/americas-tar...
141,27,2025-12-01-02-55-07 +0000,nyt,Lebanon’s Christians Look to Pope Leo’s Visit ...,https://www.nytimes.com/2025/11/30/world/middl...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
